## Obsession and Compulsion using BioBERT Model.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import time
import datetime
import gc
import random
from nltk.corpus import stopwords
import re

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler,random_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import transformers
from transformers import BertForSequenceClassification, AdamW, BertConfig,BertTokenizer,get_linear_schedule_with_warmup

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/BERT_trained/Dataset/posts_with_ontology_labe_and_negative.csv")
df.head()

,post,prefLabel,compulsion,obsession,expand1
0,Hi -\nSo I haven't been on here since December...,"['depression', 'weight gain', 'Medication', 'a...",0,1,"Aggressive obsession, Contamination obsession,..."
1,"Hi all, hope you're all having a wonderful ban...","['compulsion', 'anger', 'symptom', 'compassion...",1,1,"Aggressive intrusive thoughts, Contamination i..."
2,"Hi, \nFirst, I hope everyone managed to have s...","['hope', 'happiness', 'guilt', 'fear', 'obsess...",0,1,"Aggressive obsession, Contamination obsession,..."
3,Hello everyone. I could really use your help r...,"['Treatment', 'hope', 'Thought', 'obsession', ...",0,1,"Aggressive obsession, Contamination obsession,..."
4,"Though it comes in many flavors, one of the mo...","['quality', 'Intrusive thoughts', 'fall', 'beh...",1,1,"Aggressive obsession, Contamination obsession,..."


In [ ]:
import nltk
nltk.download('stopwords')

sw = stopwords.words('english')

def clean_text(text):

    text = str(text).lower()

    text = re.sub(r"[^a-zA-Z?.!,¿]+", " ", text) # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")

    text = re.sub(r"http\S+", "",text) #Removing URLs
    #text = re.sub(r"http", "",text)

    html=re.compile(r'<.*?>')

    text = html.sub(r'',text) #Removing html tags

    punctuations = '@#!?+&*[]-%.:/();$=><|{}^' + "'`" + '_'
    for p in punctuations:
        text = text.replace(p,'') #Removing punctuations

    text = [word.lower() for word in text.split() if word.lower() not in sw]

    text = " ".join(text) #removing stopwords

    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text) #Removing emojis
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df['post'] = df['post'].apply(lambda x: clean_text(x))

In [ ]:
#######Considering compulsion#######################
posts = df.post.values
labels = df.compulsion.values

In [ ]:
#!pip install accelerate -U
#!pip install transformers[torch]
#!pip install bitsandbytes
torch.cuda.is_available()

True

In [ ]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained('dmis-lab/biobert-base-cased-v1.1') #"emilyalsentzer/Bio_ClinicalBERT")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

In [ ]:
max_len = 0
count = 0
# For every sentence...
for sent in posts:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))
    if len(input_ids) < 128:
      count +=1

print('Max sentence length: ', max_len, count)


Max sentence length:  1459 6119


In [ ]:
input_ids = []
attention_masks = []
max_len = 128 # max_len if max_len <= 512 else 512
# For every post...
for post in posts:
    # `encode_plus` will: Tokenize the sentence, Prepend the `[CLS]` token to the start, Append the `[SEP]` token to the end.
    #   Map tokens to their IDs, Pad or truncate the sentence to `max_length, Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        post,                    # Sentence to encode.
                        truncation = True,
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = max_len,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )

    # Add the encoded sentence to the list.
    input_ids.append(encoded_dict['input_ids'])

    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', posts[0])
print('Token IDs:', input_ids[0])

Original:  hi since december since recovering prozac since depression diagnosed ocd january psych bumped higher dosage prozac mg great job wanted go making gain weight freaking ocd revolves around cheating like knowand recent obsession put quotes around cuz question whether obsession real well let start beginning went fianc friends place drinks played card games fiance whole time except went bathroom remember everything night despite alcohol system came home continued drinking fiance eventually blacked ended dream cheating partner someone seen earlier night well really cheating taking guys hands placing odd place image head nothing else really remember waking realize dream, think stirred went right back sleep dont even remember keep thinking dream actually happened fiance keeps telling whole time nothing happened think dream image day driving nuts always question whether really ocd doc diagnosed make feel better everything true real appt psych tomorrow think going write script zoloftap

<ipython-input-13-872311266200>:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels)


In [ ]:
# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)

# Create a 80-15-5 train-validation-test split.

# Calculate the number of samples to include in each set.
train_size = int(0.8 * len(dataset))
temp_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, temp_dataset = random_split(dataset, [train_size, temp_size])

val_size = int(temp_size * 0.75)
tes_size = int(temp_size * 0.25)

val_dataset, tes_dataset = random_split(temp_dataset, [val_size, tes_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))
print('{:>5,} test samples'.format(tes_size))

7,597 training samples
1,425 validation samples
  475 test samples


In [ ]:
# The DataLoader needs to know our batch size for training, so we specify it
# here. For fine-tuning BERT on a specific task, the authors recommend a batch
# size of 16 or 32.
batch_size = 16 #recommended in BioBERT is 32 but this coudl result in memory full

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order.
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
            )

In [ ]:
from transformers import BertForSequenceClassification
# Load BertForSequenceClassification, the pretrained BioBERT model with a single
# linear classification layer on top.
model = BertForSequenceClassification.from_pretrained('dmis-lab/biobert-base-cased-v1.1', num_labels = 2, output_attentions = False, output_hidden_states = False, )
# if device == "cuda:0":
# # Tell pytorch to run this model on the GPU.
#     model = model.cuda()
model = model.to(device)

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
optimizer = torch.optim.AdamW(model.parameters(),
                  lr = 0.1e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8, # args.adam_epsilon  - default is 1e-8.
                  weight_decay=0.01
                )

In [ ]:
# Number of training epochs. The BioBERT authors recommend between 2 and 4.
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 4

# Total number of training steps is [number of batches] x [number of epochs].
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = total_steps)

In [ ]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
def format_time(elapsed):
    #Takes a time in seconds and returns a string hh:mm:ss
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================
    # Perform one full pass over the training set.
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    # Measure how long the training epoch takes.
    t0 = time.time()
    total_train_loss = 0
    model.train()
    for step, batch in enumerate(train_dataloader):
        # Unpack this training batch from our dataloader.
        # As we unpack the batch, we'll also copy each tensor to the device using  `to` method.
        # `batch` contains three pytorch tensors:
        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        optimizer.zero_grad()
        output = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        loss = output.loss
        total_train_loss += loss
        # Perform a backward pass to calculate the gradients.
        loss.backward()
        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()
        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)

    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on our validation set.
    print("")
    print("Running Validation...")
    t0 = time.time()
    # Put the model in evaluation mode--the dropout layers behave differently during evaluation.
    model.eval()
    # Tracking variables
    total_eval_accuracy = 0
    best_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0
    # Evaluate data for one epoch
    for batch in validation_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        # Tell pytorch not to bother with constructing the compute graph during forward pass, since this is only needed for backprop (training).
        with torch.no_grad():
            output= model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        loss = output.loss
        total_eval_loss += loss.item()
        # Move logits and labels to CPU if we are using GPU
        logits = output.logits
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        # Calculate the accuracy for this batch of test sentences, and accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))
    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    if avg_val_accuracy > best_eval_accuracy:
        torch.save(model, 'bert_model')
        best_eval_accuracy = avg_val_accuracy
    #print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    #print("  Validation took: {:}".format(validation_time))
    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        })
print("")
print("Training complete!")
print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))



======== Epoch 1 / 4 ========
Training...

  Average training loss: 0.60
  Training epcoh took: 0:02:40

Running Validation...
  Accuracy: 0.74

======== Epoch 2 / 4 ========
Training...

  Average training loss: 0.50
  Training epcoh took: 0:02:50

Running Validation...
  Accuracy: 0.78

======== Epoch 3 / 4 ========
Training...

  Average training loss: 0.36
  Training epcoh took: 0:02:54

Running Validation...
  Accuracy: 0.94

======== Epoch 4 / 4 ========
Training...

  Average training loss: 0.23
  Training epcoh took: 0:02:54

Running Validation...
  Accuracy: 0.94

Training complete!
Total training took 0:12:09 (h:mm:ss)


In [ ]:
####saving the model in the drive
model = torch.load('bert_model')
torch.save(model, '/content/drive/MyDrive/BERT_trained/Models/BioBERT_model_compulsion')

In [ ]:
#######loading the model from the drive
model = torch.load('/content/drive/MyDrive/BERT_trained/Models/BioBERT_model_compulsion', map_location=torch.device('cpu'))

In [ ]:
#########Testing the BioBert fine-tune model with a test dataset
df_test = df.iloc[tes_dataset.indices]
df_test

,post,prefLabel,compulsion,obsession,expand1
921,"hello everyone, ocd life first time ever sort ...","['site', 'email', 'anxiety', 'anxiety', 'Thoug...",0,1,"Aggressive obsession, Contamination obsession,..."
1720,"hello all, hope everyone coping well wanted sh...","['hope', 'OCD', 'Thought', 'obsession', 'compu...",1,1,"Aggressive obsession, Contamination obsession,..."
868,"hi, ocd flared lockdown aware must resist comp...","['OCD', 'compulsion', 'obsession']",1,1,"Aggressive obsession, Contamination obsession,..."
129,housework twice week normally find anxiety pro...,"['compulsion', 'outside', 'OCD', 'anxiety', 'a...",1,0,"morbid thoughts, Mental image, Homicidal thoug..."
1856,diagnosed ocd think may come part parcel anxie...,"['obsession', 'Intrusive thoughts', 'guilt', '...",0,1,"Aggressive obsession, Contamination obsession,..."
...,...,...,...,...,...
539,recently wrote particular violent intrusive th...,"['Intrusive thoughts', 'Thought']",0,1,"Aggressive intrusive thoughts, Contamination i..."
1532,"hi guys, wanted give update story feel quite p...","['OCD', 'obsession', 'rash', 'Thought', 'urge']",0,1,"Aggressive obsession, Contamination obsession,..."
638,"hi havent posted quite well, anxiety low prese...","['Intrusive thoughts', 'Thought', 'anxiety', '...",0,1,"Aggressive intrusive thoughts, Contamination i..."
935,currently going cold turkey giving mental reas...,"['compulsion', 'checking']",1,0,"Compulsive behavior, Compulsion"


In [ ]:
test_posts = df_test['post'].values

In [ ]:
max_len = 128
test_input_ids = []
test_attention_masks = []
for post in test_posts:
    encoded_dict = tokenizer.encode_plus(
                        post,
                        add_special_tokens = True,
                        truncation = True,
                        max_length = max_len,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt',
                   )
    test_input_ids.append(encoded_dict['input_ids'])
    test_attention_masks.append(encoded_dict['attention_mask'])
test_input_ids = torch.cat(test_input_ids, dim=0)
test_attention_masks = torch.cat(test_attention_masks, dim=0)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2645: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
batch_size = 16
test_dataset = TensorDataset(test_input_ids, test_attention_masks)
test_dataloader = DataLoader(
            test_dataset, # The validation samples.
            sampler = SequentialSampler(test_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [ ]:
predictions = []
for batch in test_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        with torch.no_grad():
            output= model(b_input_ids,
                                   token_type_ids=None,
                                   attention_mask=b_input_mask)
            logits = output.logits
            logits = logits.detach().cpu().numpy()
            pred_flat = np.argmax(logits, axis=1).flatten()

            predictions.extend(list(pred_flat))

In [ ]:
#predictions
df_exp_mod = pd.DataFrame()
df_exp_mod['post'] = df_test['post']
df_exp_mod['compulsion-test'] = df_test['compulsion']
df_exp_mod['compulsion-BioBert-model-pred'] = predictions
df_exp_mod.head()

,post,compulsion-test,compulsion-BioBert-model-pred
921,"hello everyone, ocd life first time ever sort ...",0,0
1720,"hello all, hope everyone coping well wanted sh...",1,1
868,"hi, ocd flared lockdown aware must resist comp...",1,1
129,housework twice week normally find anxiety pro...,1,1
1856,diagnosed ocd think may come part parcel anxie...,0,0


In [ ]:
print(classification_report(df_exp_mod['compulsion-test'], df_exp_mod['compulsion-BioBert-model-pred']))

              precision    recall  f1-score   support

           0       0.91      0.99      0.95       280
           1       0.99      0.85      0.91       195

    accuracy                           0.93       475
   macro avg       0.95      0.92      0.93       475
weighted avg       0.94      0.93      0.93       475



In [ ]:
##########Confusion matrix########################3
import numpy as np
import matplotlib.pyplot as plt

pred_labels = df_exp_mod['compulsion-BioBert-model-pred']
true_labels = df_exp_mod['compulsion-test']

#TP is True Positive: predict a label of 1 (positive), and the true label is 1.
TP = np.sum(np.logical_and(pred_labels == 1, true_labels == 1))
# TN: predict a label of 0 (negative), and the true label is 0.
TN = np.sum(np.logical_and(pred_labels == 0, true_labels == 0))
# FP: predict a label of 1 (positive), but the true label is 0.
FP = np.sum(np.logical_and(pred_labels == 1, true_labels == 0))
# FN: predict a label of 0 (negative), but the true label is 1.
FN = np.sum(np.logical_and(pred_labels == 0, true_labels == 1))
print('TP: %i, FP: %i, TN: %i, FN: %i' % (TP,FP,TN,FN))

TP: 166, FP: 2, TN: 278, FN: 29


In [ ]:
training_stats

[{'epoch': 1,
  'Training Loss': tensor(0.6000, device='cuda:0', grad_fn=<DivBackward0>),
  'Valid. Loss': 0.5224721388684379,
  'Valid. Accur.': 0.7375,
  'Training Time': '0:02:40',
  'Validation Time': '0:00:11'},
 {'epoch': 2,
  'Training Loss': tensor(0.4974, device='cuda:0', grad_fn=<DivBackward0>),
  'Valid. Loss': 0.4325342155165143,
  'Valid. Accur.': 0.7784722222222222,
  'Training Time': '0:02:50',
  'Validation Time': '0:00:11'},
 {'epoch': 3,
  'Training Loss': tensor(0.3615, device='cuda:0', grad_fn=<DivBackward0>),
  'Valid. Loss': 0.2376611418194241,
  'Valid. Accur.': 0.9368055555555556,
  'Training Time': '0:02:54',
  'Validation Time': '0:00:11'},
 {'epoch': 4,
  'Training Loss': tensor(0.2286, device='cuda:0', grad_fn=<DivBackward0>),
  'Valid. Loss': 0.213240085542202,
  'Valid. Accur.': 0.94375,
  'Training Time': '0:02:54',
  'Validation Time': '0:00:11'}]

In [ ]:
########################BioBert model fine-tune for compulsion ################################################################################